# Lab02: Web Crawler (Continue).

- MSSV: 1712429
- Họ và tên:Đỗ Văn Hiệp

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Bạn nộp file notebook với tên `MSSV.ipynb` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên file là `1234567.ipynb`) và nộp file này trên moodle.

**Nội dung bài tập**

Thu thập và thể hiện dữ liệu web.

## 2. Cài đặt

### 2.1. Import library

In [1]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string

### 2.2. HTML Parser


Bộ phân tích cú pháp HTML (HTML Parser): nhận HTML code và trích xuất thông tin liên quan như tiêu đề của trang, đoạn văn trong trang, tiêu đề trong trang, liên kết, văn bản in đậm, v.v.

In [2]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>')

Start tag: html
Start tag: head
Start tag: title
Data  : Test
End tag : title
End tag : head


#### Loại bỏ các HTML tag không cần thiết bằng cách thiết lập filter

In [3]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

### 2.3 Thu thập nội dung trang Web

#### Trong bài tập này ta thể hiện tài liệu (hay nội dung trang web) với cấu trúc đơn giản như sau: 
- Gọi $D$ là một tập tài liệu chứa *n* tài liệu: $D=\left\{d_1,d_2,...,d_n\right\}$.
- Ta thể hiện tài liệu bằng một dictionary `data={}` với `data[word]=[[url_idx_1,url_idx_2,...],frequency]` với `url_index`$\in{\left[{1,n}\right]}$

VD: `data['at']=[[1,2], 5]`
Từ `at` xuất hiện trong đường dẫn có index `1` và `2` tổng số lần xuất hiện là 5.

#### Bước 1: liệt kê các từ xuất hiện trong trang web:

In [4]:
def wordList(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []
    
    for i in range(0,len(filtered_text)):
        if filtered_text[i] in string.punctuation:
            filtered_text[i]= filtered_text[i].replace(filtered_text[i]," ")#replace " "
    for i in range(0,len(filtered_text)):
        list_w_split=[]
        list_w_split=filtered_text[i].split(" ")
        for j in range(0,len(list_w_split)):
            word_list.append(list_w_split[j])
    for i in word_list:
        if i in string.punctuation:
            word_list.remove(i)
    return word_list
  
# Test
print(wordList('https://en.wikipedia.org/wiki/Web_mining'))


['Web', 'mining', 'From', 'Wikipedia,', 'the', 'free', 'encyclopedia', 'This', 'article', 'includes', 'a', 'related', 'reading', 'or', 'but', 'its', 'sources', 'remain', 'unclear', 'because', 'it', 'lacks', 'October', '2020', 'This', 'article', 'may', 'require', 'The', 'specific', 'problem', 'is:', 'The', 'article', 'needs', 'sufficient', 'inline', 'references,', 'and', 'a', 'complete', 're-write', 'has', 'been', 'proposed.', 'October', '2020', 'Web', 'mining', 'As', 'the', 'name', 'proposes,', 'this', 'is', 'information', 'gathered', 'by', 'mining', 'the', 'web.', 'It', 'makes', 'utilization', 'of', 'automated', 'apparatuses', 'to', 'reveal', 'and', 'extricate', 'data', 'from', 'servers', 'and', 'web2', 'reports,', 'and', 'it', 'permits', 'organizations', 'to', 'get', 'to', 'both', 'organized', 'and', 'unstructured', 'information', 'from', 'browser', 'activities,', 'server', 'logs,', 'website', 'and', 'link', 'structure,', 'page', 'content', 'and', 'different', 'sources.\n', 'The', 'g

#### Bước 2: Tính tần suất xuất hiện của từ trong 1 trang web, lưu trữ dữ liệu vào data:

In [5]:
def read_url(url, url_idx, data):
    word_list = wordList(url)
    print("length of word",len(word_list))
    # TODO
    # Với mỗi từ w trong word_list:
    #   Nếu w chưa có trong data thì khởi tạo data[w] = [[url_idx], 1]
    #   Ngược lại thì thêm url_idx vào data[w][0] (nếu chưa có) và tăng data[w][1] lên 1 đơn vị
    for i in range(0,len(word_list)) :
        if word_list[i] not in data.keys():
            data[word_list[i]]=[[url_idx],1]
            
            
        else:
            #check url
            if url_idx in data[word_list[i]][0]:
                data[word_list[i]][0].append(i)
                
            data[word_list[i]][1]=data[word_list[i]][1]+1
            
        

# Test
test_data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, test_data)
test_data

length of word 1109


{'Web': [[1,
   47,
   106,
   116,
   119,
   122,
   135,
   152,
   167,
   184,
   193,
   197,
   205,
   209,
   213,
   231,
   234,
   244,
   248,
   251,
   262,
   270,
   273,
   276,
   292,
   295,
   310,
   319,
   321,
   337,
   351,
   428,
   619,
   641,
   645,
   660,
   663,
   679,
   746,
   812,
   830,
   890,
   915,
   939,
   970],
  45],
 'mining': [[1,
   48,
   58,
   108,
   124,
   142,
   174,
   194,
   199,
   207,
   211,
   215,
   232,
   235,
   246,
   250,
   253,
   263,
   268,
   294,
   297,
   323,
   353,
   394,
   430,
   561,
   595,
   621,
   643,
   647,
   657,
   662,
   665,
   736,
   748,
   813,
   832,
   861],
  38],
 'From': [[1], 1],
 'Wikipedia,': [[1], 1],
 'the': [[1,
   50,
   59,
   115,
   132,
   148,
   259,
   299,
   305,
   331,
   415,
   426,
   462,
   474,
   477,
   485,
   495,
   523,
   532,
   543,
   568,
   571,
   576,
   592,
   616,
   659,
   667,
   685,
   766,
   774,
   777,
   783,
   801,

#### Bước 3: Chạy chương trình lưu kết quả vào file output.txt

In [6]:
data = {}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1, data)
read_url('https://en.wikipedia.org/wiki/Data_mining', 2, data)

sorted_keys = sorted(data.keys())

with open("output.txt", "w", encoding="utf-8") as f:
    output_line = "Word".ljust(20) + "Frequency".ljust(15) + "URL_idx".ljust(15) + "\n"
    f.writelines(output_line)
    f.writelines('---------------------------------------------------------------------\n\n')
    for key in sorted_keys:
        output_string = str(key).ljust(20) + str(data[key][1]).ljust(15) + str(data[key][0]).ljust(15) + "\n"
        f.writelines(output_string)


length of word 1109
length of word 2363


In [7]:
# TODO
# Keyword tìm kiếm: python pickle
# Dùng pickle để lưu dictionary data xuống đĩa. Sau đó đọc lên và in ra 10 giá trị đầu tiên của nó.

# 1. Lưu data
import pickle
fileoutput = open("output.pickle", "wb")
pickle.dump(data,fileoutput)

# 2. Đọc lên và in ra
res={}
with open("output.pickle", "rb") as f:
    res= pickle.load(f)
#in ra 10 gia tri dau tien
i=0
for key, value in res.items():
    print (key, ' ', value)
    i=i+1
    if i==9:
        break

Web   [[1, 47, 106, 116, 119, 122, 135, 152, 167, 184, 193, 197, 205, 209, 213, 231, 234, 244, 248, 251, 262, 270, 273, 276, 292, 295, 310, 319, 321, 337, 351, 428, 619, 641, 645, 660, 663, 679, 746, 812, 830, 890, 915, 939, 970], 46]
mining   [[1, 48, 58, 108, 124, 142, 174, 194, 199, 207, 211, 215, 232, 235, 246, 250, 253, 263, 268, 294, 297, 323, 353, 394, 430, 561, 595, 621, 643, 647, 657, 662, 665, 736, 748, 813, 832, 861], 82]
From   [[1], 3]
Wikipedia,   [[1], 2]
the   [[1, 50, 59, 115, 132, 148, 259, 299, 305, 331, 415, 426, 462, 474, 477, 485, 495, 523, 532, 543, 568, 571, 576, 592, 616, 659, 667, 685, 766, 774, 777, 783, 801, 816, 823, 844, 848, 1062], 125]
free   [[1], 2]
encyclopedia   [[1], 2]
This   [[1, 24, 138, 170, 368, 519, 622, 632], 12]
article   [[1, 25, 33], 3]
